<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_dark_energy_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_dark_energy_ai.py

Physics-informed neural network for dark energy optimization,
all in double precision with normalized physics residual
and corrected plot_uncertainty implementation.
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Double-Precision Synthetic Dataset
# ------------------------------------------------------------------------------
class DarkEnergyDataset(Dataset):
    def __init__(self, n_samples=6000, seed=0):
        np.random.seed(seed)
        vd = np.random.uniform(1e-10, 1e-8,  (n_samples,1)).astype(np.float64)
        cc = np.random.uniform(1e-52, 1e-50,  (n_samples,1)).astype(np.float64)
        qf = np.random.uniform(1e-5,   1e-3,   (n_samples,1)).astype(np.float64)
        R  = np.random.uniform(-1e-35, 1e-35,  (n_samples,1)).astype(np.float64)
        md = np.random.uniform(1e-27,  1e-25,  (n_samples,1)).astype(np.float64)
        rd = np.random.uniform(1e-30,  1e-28,  (n_samples,1)).astype(np.float64)

        X_raw = np.hstack([vd, cc, qf, R, md, rd])

        sf = vd / (cc + 1e-60)
        ce = 1.0 / (1.0 + qf * vd * 1e10)
        ei = (md + rd) / (vd + 1e-60)
        Y_raw = np.hstack([sf, ce, ei])

        # add small noise
        Y_raw += 0.02 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # compute normalization stats
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + 1e-12
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + 1e-12

        X_norm = (X_raw - self.X_mean) / self.X_std
        Y_norm = (Y_raw - self.Y_mean) / self.Y_std

        self.X = torch.from_numpy(X_norm).double()
        self.Y = torch.from_numpy(Y_norm).double()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


# ------------------------------------------------------------------------------
# 2. Double-Precision Model Definition
# ------------------------------------------------------------------------------
class DarkEnergyAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dims=(128, 64), output_dim=3, p_drop=0.1):
        super().__init__()
        layers, dim = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop),
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


# ------------------------------------------------------------------------------
# 3. Normalized Physics-Informed Residual
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    # denormalize inputs
    X_den = X * stats['X_std'] + stats['X_mean']
    vd, cc = X_den[:, 0], X_den[:, 1]

    # true stability factor
    sf_true = vd / (cc + 1e-60)
    # normalize sf_true
    sf_true_norm = (sf_true - stats['Y_mean'][0]) / stats['Y_std'][0]

    # compare normalized predictions to normalized target
    sf_pred_norm = pred[:, 0]
    return torch.mean((sf_pred_norm - sf_true_norm) ** 2)


def total_loss(pred, true, X, stats, lam=1.0):
    mse_loss = nn.MSELoss()(pred, true)
    phys_loss = physics_residual(pred, X, stats)
    return mse_loss + lam * phys_loss, mse_loss, phys_loss


# ------------------------------------------------------------------------------
# 4. MC-Dropout for Uncertainty Quantification
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    arr = torch.stack(preds, dim=0)
    return arr.mean(0), arr.std(0)


# ------------------------------------------------------------------------------
# 5. Training Loop with Early Stopping & Scheduler
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-3, wd=1e-5, lam=1.0, epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for ep in range(1, epochs + 1):
        model.train()
        t_loss = 0.0
        for Xb, Yb in train_loader:
            Xb, Yb = Xb.to(device), Yb.to(device)
            pred = model(Xb)
            loss, _, _ = total_loss(pred, Yb, Xb, stats)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            t_loss += loss.item() * Xb.size(0)
        t_loss /= len(train_loader.dataset)

        model.eval()
        v_loss = 0.0
        with torch.no_grad():
            for Xb, Yb in val_loader:
                Xb, Yb = Xb.to(device), Yb.to(device)
                pred = model(Xb)
                loss, _, _ = total_loss(pred, Yb, Xb, stats)
                v_loss += loss.item() * Xb.size(0)
        v_loss /= len(val_loader.dataset)

        scheduler.step(v_loss)
        history['train'].append(t_loss)
        history['val'].append(v_loss)
        print(f"Epoch {ep:03d} | Train {t_loss:.4e} | Val {v_loss:.4e}")

        if v_loss + 1e-8 < best_val:
            best_val, wait = v_loss, 0
            torch.save(model.state_dict(), "best_dark_energy_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {ep}")
                break

    model.load_state_dict(torch.load("best_dark_energy_ai.pth"))
    return history


# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_losses(hist):
    plt.figure()
    plt.plot(hist['train'], label='Train')
    plt.plot(hist['val'],   label='Val')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


def plot_scatter(y_true, y_pred, title):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.title(title)
    plt.xlabel("True")
    plt.ylabel("Pred")
    plt.show()


def plot_uncertainty(model, stats, device):
    # 1) build vd-cc grid
    vd_vals = torch.linspace(1e-10, 1e-8, 100,
                             device=device, dtype=torch.double)
    cc_vals = torch.linspace(1e-52, 1e-50, 100,
                             device=device, dtype=torch.double)
    VD, CC = torch.meshgrid(vd_vals, cc_vals, indexing='xy')
    vd_flat, cc_flat = VD.reshape(-1), CC.reshape(-1)

    # 2) other features at their mean
    rest_mean = stats['X_mean'][2:]                               # shape [4]
    rest = rest_mean.unsqueeze(0).repeat(vd_flat.shape[0], 1)     # shape [N,4]

    # 3) assemble & standardize
    Xg = torch.cat([vd_flat.unsqueeze(1),
                    cc_flat.unsqueeze(1),
                    rest], dim=1)
    Xn = (Xg - stats['X_mean']) / stats['X_std']

    # 4) MC-dropout uncertainty
    _, std = mc_dropout_predict(model, Xn, T=100)
    std_map = std[:, 0].reshape(VD.shape).cpu().numpy()

    # 5) plot
    plt.figure()
    plt.pcolormesh(VD.cpu().numpy(),
                   CC.cpu().numpy(),
                   std_map,
                   cmap='viridis',
                   shading='auto')
    plt.colorbar(label="Std of Stability Factor")
    plt.xlabel("Vacuum Density")
    plt.ylabel("Cosmological Constant")
    plt.title("Stability Uncertainty Heatmap")
    plt.show()


# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ds = DarkEnergyDataset()
    n_val = int(0.2 * len(ds))
    train_ds, val_ds = random_split(ds, [len(ds) - n_val, n_val])
    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=256)

    stats = {
        'X_mean': torch.tensor(ds.X_mean, dtype=torch.double, device=device),
        'X_std':  torch.tensor(ds.X_std,  dtype=torch.double, device=device),
        'Y_mean': torch.tensor(ds.Y_mean, dtype=torch.double, device=device),
        'Y_std':  torch.tensor(ds.Y_std,  dtype=torch.double, device=device),
    }

    model = DarkEnergyAI().double().to(device)
    history = train(model, train_loader, val_loader, stats, device,
                    lr=1e-3, wd=1e-5, lam=1.0, epochs=100, patience=10)

    plot_losses(history)

    with torch.no_grad():
        X_all = ds.X.to(device)
        Y_pred_norm = model(X_all).cpu().numpy()
    Y_pred = Y_pred_norm * ds.Y_std + ds.Y_mean
    Y_true = ds.Y.cpu().numpy() * ds.Y_std + ds.Y_mean

    names = ["Stability Factor", "Control Efficiency", "Expansion Index"]
    for i, nm in enumerate(names):
        plot_scatter(Y_true[:, i], Y_pred[:, i], nm)

    plot_uncertainty(model, stats, device)